<a href="https://colab.research.google.com/github/PeterNaggschga/Letter-Variations-in-First-Names-IS/blob/main/LetterVariationsFirstNames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RadixTree
Implementation eines RadixTrees.

In [2]:
class RadixTree:
    """contains a dict, which links to multiple RadixTree's. The key to another tree is a str and
    if a transition ends in a word, the isWord-variable is True
    """

    def __init__(self, isWord=False, transitions=None):
        if transitions is None:
            transitions = dict()
        self.isWord = isWord
        self.transitions = transitions

    def insertWord(self, word):
        """inserts a word into the tree

        Args:
            word (str): is the str, that gets inserted
        """
        for i in range(0, len(word)):
            # goes through 'tobi' with 'tobi', 'tob', 'to', 't'
            possibleTransition = word[:len(word) - i]
            if self.transitions.get(possibleTransition) is not None:
                child = self.transitions.get(possibleTransition)
                if possibleTransition == word:
                    child.isWord = True
                else:
                    child.insertWord(word[len(possibleTransition):])
                return

            for key in self.transitions.keys():
                if possibleTransition == key[:len(possibleTransition)]:
                    child = self.transitions.pop(key)
                    newDict = dict()
                    newDict[key[len(possibleTransition):]] = child
                    self.transitions[possibleTransition] = RadixTree(possibleTransition == word, newDict)
                    if possibleTransition != word:
                        self.transitions[possibleTransition].insertWord(word[len(possibleTransition):])
                    return

        self.transitions[word] = RadixTree(True, dict())

    def __strRecursive__(self, timesOfIndentation, lengthOfTransitionString):
        result = ""
        if self.isWord:
            result += "."
        keys = list(self.transitions.keys())
        keys.sort()
        for key in keys:
            recursiveResult = self.transitions[key].__strRecursive__(timesOfIndentation + lengthOfTransitionString,
                                                                     len(key))
            result += "\n" + (timesOfIndentation + lengthOfTransitionString) * "_" + key + recursiveResult
        return result

    def __str__(self):
        """generates a readable str, containing all class variables (the tree).

        Returns:
            str: the generated str
        """
        return self.__strRecursive__(0, 0)

    def getSimilarWordsOfSameLength(self, maximumDifferentLetters, word):
        """compares the given word with entries of the same length. Returns all of them with less or equal different letters than with maximumDifferentLetters described

        Args:
            maximumDifferentLetters (int): limits the amount of accepted different letters when comparing 2 words
            word (str): the given word

        Returns:
            list: returns a list of similar words with the same length. Does contain itself
        """
        if word == "":
            return [word] if self.isWord else []

        resultList = list()
        for key in self.transitions.keys():
            if len(key) > len(word):
                continue
            differences = 0
            for i in range(0, len(key)):
                differences += word[i] != key[i]
                if differences > maximumDifferentLetters:
                    break

            if differences > maximumDifferentLetters:
                continue
            resultTmp = self.transitions[key].getSimilarWordsOfSameLength(maximumDifferentLetters - differences,
                                                                          word[len(key):])
            if resultTmp:
                resultList.extend([key + tmp for tmp in resultTmp])
        return resultList


# RadixTreesByWordLength
Ein Container, der Wörter je nach Länge in unterschiedliche RadixTrees ablegt und ansonsten wie ein Tree agiert.

In [3]:
class RadixTreesByWordLength:
    """contains a dict, which links to multiple RadixTree's, each storing words of the same length
    """

    def __init__(self):
        self.radixTrees = dict()

    def insertWord(self, word):
        """inserts a word into the tree

        Args:
            word (str): is the str, that gets inserted
        """
        length = len(word)
        if self.radixTrees.get(length) is None:
            self.radixTrees[length] = RadixTree()
        self.radixTrees[length].insertWord(word)

    def __str__(self):
        """generates a readable str, containing all class variables (the tree)

        Returns:
            str: the generated str
        """
        tmp = ""
        lengthStr = self.radixTrees.keys()
        sortedLengths = [int(lengths) for lengths in lengthStr]
        sortedLengths.sort()

        for length in sortedLengths:
            tmp += "RadixTree with words of length " + str(length) + ":"
            tmp += self.radixTrees[length].__str__()
            tmp += "\n"
        return tmp

    def getSimilarWordsOfSameLength(self, maximumDifferentLetters, word):
        """compares the given word with entries of the same length. Returns all of them with less or equal different letters than with maximumDifferentLetters described

        Args:
            maximumDifferentLetters (int): limits the amount of accepted different letters when comparing 2 words
            word (str): the given word

        Returns:
            list: returns a list of similar words with the same length. Does contain itself
        """
        length = len(word)
        if self.radixTrees.get(length) is None:
            return []
        return self.radixTrees[length].getSimilarWordsOfSameLength(maximumDifferentLetters, word)


# Name Extraction

In [5]:
#import re
# install and import Entrez and Medline first
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    from Bio import Entrez, Medline
except:
    # One of these 2 lines should work
    # !pip install Bio
    install('Bio')
from Bio import Entrez, Medline


def getPapers(myQuery, maxPapers, myEmail="freytag64@gmail.com"):
    """retrieves some Papers from Pubmed

    Args:
        myQuery (str): is the given Query 
        maxPapers (int): is a limit of the number of papers, which will be retrieved
        myEmail (str, optional): an email. Defaults to "freytag64@gmail.com".

    Returns:
        list: papers as list of dictionarys containing abstract, authors, ...
    """
    # Get articles from PubMed
    Entrez.email = myEmail
    record = Entrez.read(Entrez.esearch(db="pubmed", term=myQuery, retmax=maxPapers))
    idlist = record["IdList"]
    print("\nThere are %d records for %s." % (len(idlist), myQuery.strip()))
    records = Medline.parse(Entrez.efetch(db="pubmed", id=idlist, rettype="medline", retmode="text"))
    return list(records)


def retrieveAllFirstNames(records):
    """takes list of papers (each is a dict) and extracts the first names as a combined list with a regular expression 

    Args:
        records (list): list of papers

    Returns:
        list: list of first names as str's
    """
    # retrieves first names from authors with regular expressions
    firstNameList = list()
    for record in filter(lambda x: 'FAU' in x, records):
        for fullName in record['FAU']:
            try:
                # since names are formatted like 'Lastname, Firstnames', split by ',' to get Firstnames
                firstName = fullName.split(',')[1].strip().lower()
                names1 = list(filter(lambda x: len(x) > 1, firstName.split()))
                firstNameList.extend(names1)
                # print(fullName + " --> " + str(names1))
                
                # fixed: does not get name-pairs like "le Roux, Marlene F", since the lastname 'Roux' starts with ' ' too
                # fixed (see comment): does not really get name-pairs like "Something, A Mohammed", since A is just a single letter (it ignores A as a firstname)
                # a better one can be generated by using Multiple Sequence Alignment from the lectures. Names like Al-Abehd with '-' are just added as 'Al-Abehd' too.
                # sometimes accepts stuff like 'jr'
            except:
                pass
            
            """
            expression = r' ([a-zA-Z_-][a-zA-Z_-]+)'
            names2 = re.findall(expression, fullName)
            # firstNameList.extend(names2)            
            
            if names1 != names2:
              print(fullName + " --> " + str(names1))
              print(fullName + " --> " + str(names2))
            """

    return firstNameList

#Clustering
Clustern der Namen mit einer DBSCAN-ähnlichen Methode.

Unterschiede:
- zu große Cluster werden strenger erneut geclustert (maximale Größe: 1000 Namen)
- beim Prüfen, ob ein Knoten ein Kernknoten ist, zählen bereits erkundete Knoten nicht mehr
- Namen mit Länge 2 oder kürzer werden ignoriert

Ein Kernknoten braucht mindestens 6 unbesuchte Nachbarn um als Kernknoten zu gelten.

Eliminiert nicht erreichbare Knoten.

Knoten: einzelne Namen

Knotenübergänge (ungerichtet) zwischen Namen gleicher Länge mit maximaler Hamming-Distanz 1

In [6]:
def getNames() -> list:
    """reads names from external file

    Returns:
        list: list of firstnames
    """
    print("reads in names from external list")
    names = list()
    with open("2022FirstNames", "r") as myfile:
        try:
            names = myfile.read().split()
        finally:
            myfile.close()

    print("names in total: " + str(len(names)))
    
    return names

def getClusters() -> list:
    """reads in clusters from external file

    Returns:
        list: is list of clusters (list of names)
    """
    print("reads in clusters from external list")
    clusters = list()
    with open("clusteredNames", "r") as myfile:
        try:
            clusterAsStrings = myfile.read().split("\n")
            for string in clusterAsStrings:
                clusters.append(string.split())
        finally:
            myfile.close()

    print("clusters in total: " + str(len(clusters)))
    
    return clusters
    

def Dbscan(names : list, min_samples : int, min_word_length : int, max_cluster_length : int) -> list:
    """clusters list of names with DBSCAN

    Args:
        names (list): list of names
        min_samples (int): the number of samples in a neighborhood for a point to be considered a core point
        min_word_length (int): only words with this length or longer are clustered
        max_cluster_length (int): maximum size of cluster. If found cluster are bigger, they will devided
    Returns:
        list: list of clusters (each cluster is a list of names)
    """
    
    tree = RadixTreesByWordLength()
    clusters = list()
    names = list(set(names)) #remove duplicates
    names = list(filter(lambda x : len(x) >= min_word_length, names))
    names.sort(reverse=True)
    
    tmp = list()
    for n in names:
        tmp.append(n.lower())
    names = tmp
    
    #construct the RadixTree
    for n in names:
        tree.insertWord(n)
    
    #clustering
    while len(names) > 0:
        name = names.pop()
        cluster = set()
        stack = [name]
        
        #fill cluster
        while len(stack) > 0:
            node = stack.pop()
            
            #check if name is a core-sample
            neighbors = tree.getSimilarWordsOfSameLength(1, node)
            #already found nodes are not considered as neighbors
            toBeRemoved = list()
            for n in neighbors:
                if n in cluster:
                    toBeRemoved.append(n)
                    continue
                for c in clusters:
                    if n in c:
                        toBeRemoved.append(n)
            for n in toBeRemoved:
                neighbors.remove(n) 
                
            #add new names to cluster
            if len(neighbors) >= min_samples:
                for n in filter(lambda x : x in names, neighbors):
                    names.remove(n)
                stack.extend(neighbors)
                cluster.update(set(neighbors))
        
        cluster = list(cluster)
        #add filled cluster
        if len(cluster) > 1 and len(cluster) <= max_cluster_length:
            clusters.append(cluster)
        #if a cluster is too big, cluster it again
        elif len(cluster) > max_cluster_length:
            smallerClusters = Dbscan(cluster, min_samples + 5, min_word_length, max_cluster_length)
            clusters.extend(smallerClusters)    

    return clusters
            

In [ ]:
names = getNames()

print("clustering (takes a while):")
clusters = Dbscan(names, 3, 3, 600)

print(str(len(clusters)) + " different clusters")
i = 0
for c in clusters:
    i += len(c)
print(str(i) + " names clustered")

#save in file
with open("clusteredNames", "w+") as myfile:
    try:
        tmp = ""
        for cluster in clusters:
            for name in cluster:
                tmp += name + " "
            tmp += "\n"
        myfile.write(tmp)
    finally:
        myfile.close()

# SubstitutionMatrix
Implementation einer Substitutionsmatrix, ähnlich BLOSUM. Ignoriert aktuell alle Buchstaben, die nicht im ASCII kodiert sind.

Matrixeinträge werden nach folgender Formel berechnet: 
$$ S_{i,j} = \frac{1}{\lambda_2} \log{(\lambda_1 \cdot \frac{p_{ij}}{q_i \cdot q_j})} $$

$ p_{ij} $ ... Wahrscheinlichkeit, dass (innerhalb eines Clusters) i durch j ersetzt wird \\
$ q_i, q_j $ ... Wahrscheinlichkeiten, dass i bzw. j in einem Wort auftritt \\
$ \lambda_i $ ... (frei wählbare) Skalierungsfaktoren \\
Dabei wird $ \lambda_1 $ genutzt, um die Werte in der Matrix in positive oder negative Richtung zu verschieben. 
$ \lambda_2 $ skaliert größe der Werte linear. 
Die Funktion SubstitutionMatrix.autoscale() skaliert die Werte der Matrix standardmäßig so, dass für alle Werte gilt $ S_{i,j} \in [-5, 5] $ (Abweichungen von bis zu 0.1% möglich).

In [7]:
import numpy as np
from collections import defaultdict


class SubstitutionMatrix:

    def __init__(self, names: list, clusters: list, inner_scaling: float = 100, outer_scaling: float = 1):
        """
        Initializes and calculates a Substitution-Matrix out of the given names, clustering and scaling

        :param names: list of all names used to calculate matrix
        :param clusters: list of clusters which are lists of similar names
        :param scaling: optional scaling factor
        """
        self.letters = [
            'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

        self.name_occurrences = self.calc_name_occurrences(names)

        self.letter_prob = self.calc_letter_prob()

        self.clusters = clusters

        self.inner_scaling = inner_scaling

        self.outer_scaling = outer_scaling

        self.sub_probs = self.calc_sub_probs()

        print("Calculating substitution matrix...")
        self.substitution_matrix = self.calc_substitution_matrix()
        print("done")

    def calc_name_occurrences(self, all_names: list) -> dict:
        """
        Calculates the number of duplicates for each name in all_names

        :param all_names: list of all names used to calculate the matrix
        :return: a dictionary mapping every name to its number of occurrences
        """
        print("Calculating name occurrences...")

        result = dict()
        names, count = np.unique(all_names, return_counts=True)
        for i in range(len(names)):
            result[names[i]] = count[i]

        print("done")
        return result

    def calc_letter_prob(self) -> dict:
        """
        Calculates the probability of every letter to be in a name

        :return: a dictionary mapping every letter to its probability of being in a name
        """
        print("Calculating probability of letters...")

        letter_counts = defaultdict(int)

        n = 0
        for (name, count) in self.name_occurrences.items():
            for x in {c for c in name}:
                if x in self.letters:
                    letter_counts[x] += count
            n += count

        letter_prob = defaultdict(int)
        for (x, occurrences) in letter_counts.items():
            letter_prob[x] = occurrences / n

        print("done")
        return letter_prob

    def calc_sub_probs(self) -> dict:
        """
        Calculates the probability of one letter to be substituted by another letter

        :return: a matrix with substitution probabilities for each letter
        """
        print("Calculating probability of substitutions...")

        sub_probs = dict()
        for x in self.letters:
            sub_probs[x] = dict()
            for y in self.letters:
                sub_probs[x][y] = (0, 0)  # (prob, #examples)
        
        for cluster in self.clusters:
            for i in range(len(cluster[0])):
                occurrences = defaultdict(int)
                sum_occurrences = 0
                for name in cluster:
                    n = self.name_occurrences[name]
                    letter = name[i]
                    if letter in self.letters:
                        occurrences[name[i]] += n
                        sum_occurrences += n

                for a in occurrences.keys():
                    for b in occurrences.keys():
                        (p_old, n_old) = sub_probs[a][b]
                        n_new = sum(occurrences.values())
                        p_new = occurrences[a] * occurrences[b] / n_new
                        if a != b:
                            p_new *= 2
                        sub_probs[a][b] = ((p_new + p_old) / (n_old + n_new), n_old + n_new)

        for a in self.letters:
            for b in self.letters:
                sub_probs[a][b] = sub_probs[a][b][0]
        print("done")
        return sub_probs

    def calc_substitution_matrix(self) -> dict:
        """
        Calculates the Substitution-Matrix

        :return: a matrix with substitution values for each letter
        """
        matrix = defaultdict(dict)
        min = float("inf")
        for a in self.letters:
            for b in self.letters:
                try:
                    if self.sub_probs[a][b] == 0:
                        matrix[a][b] = "min"
                    else:
                        matrix[a][b] = (self.outer_scaling ** (-1)) * np.log2(self.inner_scaling *
                            self.sub_probs[a][b] / (self.letter_prob[a] * self.letter_prob[b]))
                        if matrix[a][b] < min:
                            min = matrix[a][b]
                            #print((a, b))
                except ZeroDivisionError:
                    matrix[a][b] = "min"

        for a in matrix.keys():
            for b in matrix[a].keys():
                if matrix[a][b] == "min":
                    #print((a, b))
                    matrix[a][b] = min
        
        return matrix

    def autoscale(self, exp_bounds: float = 5, derivation: float = 0.001):
        print(f"Scaling automatically for bounds {-exp_bounds} +/- {derivation} and {exp_bounds} +/- {derivation}...")
        (min, max) = self.get_bounds()
        print(f"Old scalings (l1, l2) = {(self.inner_scaling, self.outer_scaling)}")
        
        correction = self.inner_scaling / 2
        direction = 1 if abs(min) > abs(min) else -1
        while abs(abs(max) - abs(min)) > derivation:
            new_scaling = self.inner_scaling + direction * correction
            if new_scaling <= 0:
                new_scaling = 1
            self.set_scaling(inner_scaling=new_scaling)
            
            # print((new_scaling, min, max))
            (min, max) = self.get_bounds()
            direction_new = 1 if abs(min) > abs(max) else -1
            if direction_new != direction:
                correction /= 2
                direction = direction_new
        
        (min, max) = self.get_bounds()
        outer_scaling = (max + abs(min)) / (2 * exp_bounds)
        self.set_scaling(self.inner_scaling, outer_scaling)
        print(f"(min, max) = {self.get_bounds()}")

        print(f"New scalings (l1, l2) = {(self.inner_scaling, self.outer_scaling)}")
        print("done")

    def get_bounds(self) -> ():
        """
        Returns a triple of values representing minimum, maximum and average of the substitution matrix

        :return:
        """
        vals = [self.substitution_matrix[a][b] for a in self.letters for b in self.letters]
        return (np.min(vals), np.max(vals))

    def set_names(self, names: list, clusters: list = None):
        """
        Sets a new list of names (and a new clustering) as basis and calculates the new Substitution-Matrix

        :param names: list(str)
        :param clusters: list(list(str))
        :return:
        """
        self.__init__(names, clusters if clusters else self.clusters, self.scaling)

    def set_clusters(self, clusters: list):
        """
        Sets a new clustering and calculates the new Substitution-Matrix

        :param clusters: list(list(str))
        :return:
        """
        self.clusters = clusters
        self.sub_probs = self.calc_sub_probs()
        self.substitution_matrix = self.calc_substitution_matrix()

    def set_scaling(self, inner_scaling: float = 100, outer_scaling: float = 1):
        """
        Set a new scaling and calculate the new Substitution-Matrix
        :param scaling:
        :return:
        """
        if self.inner_scaling != inner_scaling or self.outer_scaling != outer_scaling:
            self.inner_scaling = inner_scaling
            self.outer_scaling = outer_scaling
            self.substitution_matrix = self.calc_substitution_matrix()

    def __str__(self):
        tmp = "Substitutionsmatrix:\n  "
        for to in self.letters:
            tmp += "\t" + to
        tmp += "\n"

        for fr in self.letters:
            tmp += fr
            for to in self.letters:
                strEntry = str(self.substitution_matrix[fr][to])
                if len(strEntry) < 4:
                    strEntry = (4 - len(strEntry)) * "0" + strEntry
                tmp += "\t" + strEntry[:4]
            tmp += "\n"
        return tmp

In [8]:
""" Testing

[(Klaus, 10), (Claus, 5), (Klaas, 15)]
P(K - C) = 2 * (5/6 * 1/6) = 10/36
P(K - K) = 5/6 * 5/6 = 25/36
P(C - C) = 1/6 * 1/6 = 1/36

[(Klara, 2), (Clara, 4), (Clars, 4)]
P(K - C) = 2 * (1/5 * 4/5) = 8/25
P(K - K) = 1/5 * 1/5 = 1/25
P(C - C) = 4/5 * 4/5 = 16/25

Ges:
P(K - C) = 30/40 * 10/36 + 10/40 * 8/25 = 
"""
names = [("klara", 2), ("clara", 4), ("clars", 4), ("klaus", 10), ("claus", 5), ("klaas", 15)]
all_names = []
for (name, count) in names:
    for i in range(count):
        all_names.append(name)

clusters = [["klaus", "claus", "klaas"], ["klara", "clara", "clars"]]

matrix = SubstitutionMatrix(all_names, clusters)
matrix.autoscale()
print(matrix)


Calculating name occurrences...
done
Calculating probability of letters...
done
Calculating probability of substitutions...
done
Calculating substitution matrix...
done
Scaling automatically for bounds -5 +/- 0.001 and 5 +/- 0.001...
Old scalings (l1, l2) = (100, 1)
(min, max) = (-4.9997772512294585, 5.0002227487705415)
New scalings (l1, l2) = (1.1556396484375, 0.8417008237922831)
done
Substitutionsmatrix:
  	a	b	c	d	e	f	g	h	i	j	k	l	m	n	o	p	q	r	s	t	u	v	w	x	y	z
a	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-0.7	-4.9	0.74	-4.9	-4.9	-4.9	-4.9	-4.9
b	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9
c	-4.9	-4.9	0.96	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-1.3	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9
d	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9	-4.9
e	-4.9	

# Demo

In [10]:
maxPapers = 10000  # limit the number of papers retrieved
myQuery = "(\"2021/01/20\"[Date - Publication] : \"2021/01/20\"[Date - Publication])"
records = getPapers(myQuery, maxPapers)


There are 4926 records for ("2021/01/20"[Date - Publication] : "2021/01/20"[Date - Publication]).


In [11]:
names = retrieveAllFirstNames(records)

min_name_length = 4
names = list(filter(lambda x: len(x) >= min_name_length, names))

print(list(np.unique(names)))

['a-rang', 'a-reum', 'aaditya', 'aafke', 'aakanksha', 'aaliyah', 'aalt', 'aamer', 'aamir', 'aanchal', 'aarent', 'aarno', 'aaron', 'aarooran', 'aarti', 'aasems', 'aayush', 'abani', 'abate', 'abay', 'abbas', 'abbie', 'abby', 'abdalazeem', 'abdalghani', 'abdallah', 'abdel', 'abdel-karim', 'abdel-moneim', 'abdel-rahman', 'abdelahhad', 'abdelaziz', 'abdelhafid', 'abdelhamid', 'abdelkader', 'abdellatif', 'abdelmoumen', 'abdelouahed', 'abdelsattar', 'abdelwahab', 'abdenacer', 'abdenbi', 'abderrahim', 'abderraouf', 'abdessamad', 'abdolreza', 'abdou', 'abdoulaye', 'abdul', 'abdul-ilah', 'abdul-rahim', 'abdulaziz', 'abdulhak', 'abdulkadir', 'abdulkarim', 'abdullah', 'abdullatif', 'abdulmajeed', 'abdulmunem', 'abdulrahman', 'abdulwahed', 'abdurrahman', 'abdurrashid', 'abebaw', 'abelardo', 'abhas', 'abhay', 'abhijit', 'abhik', 'abhilash', 'abhilasha', 'abhinav', 'abhiram', 'abhisek', 'abhishek', 'abid', 'abigail', 'abimael', 'abina', 'abinaya', 'abiodun', 'abir', 'abolfazl', 'abou', 'aboubakr', 'a

In [ ]:
clusters = Dbscan(names, 3, min_name_length, 600)
for cluster in clusters:
    print(cluster)

In [14]:
matrix = SubstitutionMatrix(names, clusters)
matrix.autoscale()

#print(matrix.name_occurrences)
print(matrix)

Calculating name occurrences...
done
Calculating probability of letters...
done
Calculating probability of substitutions...
done
Calculating substitution matrix...
done
Scaling automatically for bounds -5 +/- 0.001 and 5 +/- 0.001...
Old scalings (l1, l2) = (100, 1)
(min, max) = (-4.999858468797423, 5.000141531202578)
New scalings (l1, l2) = (31.6396484375, 1.9948448593832193)
done
Substitutionsmatrix:
  	a	b	c	d	e	f	g	h	i	j	k	l	m	n	o	p	q	r	s	t	u	v	w	x	y	z
a	-2.4	-0.6	-4.0	-4.1	-2.7	-3.2	0.88	-0.7	-2.6	-0.8	-2.0	-1.5	-1.2	-2.8	-2.0	-0.9	0.11	-2.0	-0.9	-2.1	-1.9	-3.3	-0.5	-4.0	-1.7	-0.7
b	-0.6	-0.7	-1.0	-0.5	2.88	-0.1	-2.0	-0.3	-0.7	-0.8	-0.6	-0.0	-0.7	-1.9	1.60	0.86	0.57	-0.0	-0.1	-0.1	4.03	2.37	-1.0	1.66	0.20	0.73
c	-4.0	-1.0	-2.1	-1.6	-1.8	-0.6	-0.5	-1.6	-1.0	-1.8	1.01	-1.6	-1.7	-2.9	-3.2	0.21	0.62	-0.5	-0.7	0.34	-2.4	1.28	-1.7	-1.4	-1.3	0.88
d	-4.1	-0.5	-1.6	-1.1	-0.7	-1.0	-1.7	-0.6	-2.6	-2.0	-0.6	-1.3	-0.8	-0.6	-0.7	-0.7	0.63	-1.4	-1.2	-1.8	-4.8	1.40	-0.6	-0.1	-0.5	0.46
e	-2.7	2.88

In [15]:
sub_probs = matrix.calc_sub_probs()
print(sub_probs)
print(sum(sub_probs["c"].values()))
print(matrix.letter_prob["a"])

Calculating probability of substitutions...
done
{'a': {'a': 0.0004507288587816751, 'b': 0.0006916827772608785, 'c': 1.2064007232035658e-05, 'd': 9.752410236069308e-06, 'e': 0.0001911641484473404, 'f': 1.385586881097971e-05, 'g': 0.013203780497315201, 'h': 0.001945381902234316, 'i': 0.0002848188289746163, 'j': 0.0007402555213782692, 'k': 0.00012569208212663798, 'l': 0.0005850282112323909, 'm': 0.0006906088790927456, 'n': 0.00021242947914351147, 'o': 0.00040165115817699585, 'p': 0.00034673978419967806, 'q': 0.0005417086321757178, 'r': 0.00040726891578693404, 's': 0.001345938982837399, 't': 0.00019799482856318725, 'u': 0.00028786044481451847, 'v': 1.1260754020089185e-05, 'w': 0.0004482163557883264, 'x': 2.971864961188211e-06, 'y': 0.00026439442539033265, 'z': 0.0003544081628910067}, 'b': {'a': 0.0006916827772608785, 'b': 6.461174482617618e-05, 'c': 8.779491758670576e-05, 'd': 0.00017111448318295942, 'e': 0.0560711073874745, 'f': 0.0001192382417528421, 'g': 2.7129923440888976e-05, 'h': 0.